In [ ]:
from mudata_validator import validate_mudata

In [ ]:
# Validate a MuData object
import muon as mu
mdata = mu.read_h5mu({"example.h5amu"})
validate_mudata(mdata)

In [ ]:
# Or validate using the h5mu file path
validate_mudata("example.h5mu")